@author: Gediyon M. Girma

off policy MC controller via importance sampling for MountainCar environment

prblem: Mountain Car


*   reward: The goal is to reach the flag placed on top of the right hill as quickly as possible, as such the agent is penalized with a reward of -1 for each timestep.
*   observation: ndarray of size (2,) where index 0 is position in range of  [-1.2, 0.6]


In [35]:
import gym
import time
import itertools
import random
import numpy as np

  and should_run_async(code)



In [37]:
# loading the environment
env = gym.make('MountainCar-v0')

  and should_run_async(code)

  deprecation(

  deprecation(



In [38]:
# discretization of the observation space

position_bins = 10
velocity_bins = 10

def discretize(value,min_value,max_value,num_bins):
  """
  Discretizes a value using a uniform grid, clamping values outside of the grid range.

  Args:
    value: The value to discretize.
    min_value: The minimum value of the grid.
    max_value: The maximum value of the grid.
    num_bins: The number of bins to use.

  Returns:
    The discretized value.
  """
  # if the observed value is out of the max and min range assign it to the edge values
  value = min(max(value, min_value), max_value)

  # calculate the bin size
  bin_size = (max_value - min_value) / num_bins

  # calculate the bin index and bin index starts from 1
  bin_index = int((value - min_value) / bin_size)

  return bin_index

def discretize_observation(observation):
  """
  Discretizes the observation space.

  Args:
    observation: The observation to discretize.

  Returns:
    The discretized observation.
  """


  position, velocity = observation

  state = [discretize(position, -1.2, 0.6, position_bins - 1), # discretize the position
           discretize(velocity, -0.07, 0.07, velocity_bins - 1) # discretize the velocity
           ]
  return tuple(state)



# initializing

q = {} # action value function
c = {} # sum of weights
policy = {} # target policy
b = {} # behaviour policy

# formaulate the state space with every combination of the discritsized elements of the states
states = itertools.product(np.arange(position_bins),
                           np.arange(velocity_bins))


for state in states:
  c[state] = np.zeros((env.action_space.n)) # initialize the sum for the s-a pair
  b[state] = np.full(((env.action_space.n)),1/env.action_space.n) # initialize the behaviour policy
  q[state] = np.random.random((env.action_space.n)) # initialize the action-value function
  policy[state] = np.zeros((env.action_space.n)) # initialize target policy with zero probability for all actions

  best_action = np.argmax(q[state])  # selecting the best action based on the action-value function
  policy[state][best_action] = 1  # setting 1 to the probability of the best action

  and should_run_async(code)



In [41]:
# Iteration

start_timer = time.time() # starting the timer


epsilon = 0.1

episodes = 1e5
episode = 1
while episode < episodes:

  # reset the environment
  obs = env.reset()
  state = discretize_observation(obs) # discretize the observation

  # initializing a dictionary to save the episode's trajectory
  episode_trajectory = [[0, state]] # starting reward (0) and state

  # generating an episodes using a an epsilon greedy policy
  for t in range(1,201):

    action = np.random.choice(np.arange(env.action_space.n), p = b[state]) # selecting the action based on the behaviour policy

    episode_trajectory[-1].append(action) # appending the action to the episode trajectory

    obs, reward, done, info = env.step(action)
    state = discretize_observation(obs) # discretize the observation
    episode_trajectory.append([reward,state]) # append the reward and the state to the episode trajectory

    if done:
      break


  # initializing the return and the weight
  G = 0
  W = 1
  gamma = 0.9 # discount factor

  # updating the action-value function
  for t in range(len(episode_trajectory)-2,-1,-1):
    G = gamma*G  + episode_trajectory[t+1][0]
    state = episode_trajectory[t][1]
    action = episode_trajectory[t][2]
    c[state][action] += W  # updating the sum for the s-a pair
    q[state][action] += (W/c[state][action]) * (G - q[state][action]) # incremental averaging

    best_action = np.argmax(q[state]) # selecting the best action based on the action-value function

    policy[state] = np.zeros(env.action_space.n) # resetting the policy for the given state
    policy[state][best_action] = 1 # setting the probability of the best action to 1


    if action != best_action:
      break
    W = W/b[state][action]

  # updating the behavioural policy b to be the epsilon soft policy of the target policy
  for state in states:
    best_action = np.argmax(q[state])
    b[state] = np.full(((env.action_space.n)),epsilon/env.action_space.n)
    b[state][best_action] += 1-epsilon


  episode += 1
  if episode % 10000 == 0:
    end_timer = time.time()
    timer = end_timer - start_timer
    elapsed_time_struct = time.gmtime(timer)
    formatted_time = time.strftime("%H:%M:%S", elapsed_time_struct)
    print("Episode: ",episode, " time: ", formatted_time)

  if episode == episodes:
    print("done!")

  and should_run_async(code)

  if not isinstance(terminated, (bool, np.bool8)):



Episode:  10000  time:  00:03:56
Episode:  20000  time:  00:07:53
Episode:  30000  time:  00:11:41
Episode:  40000  time:  00:15:19
Episode:  50000  time:  00:18:57
Episode:  60000  time:  00:22:34
Episode:  70000  time:  00:26:13
Episode:  80000  time:  00:29:52
Episode:  90000  time:  00:33:32
Episode:  100000  time:  00:37:12
done!


In [47]:
obs = env.reset()
state = discretize_observation(obs)
reward = 0
for i in range(50000):
  action = np.argmax(policy[state])
  print("step",i, "reward: ", reward," state: ", state," action=",action)
  obs, reward, done, info = env.step(action)

  state = discretize_observation(obs)

  if done:
    print ("done")
    break

# ipythondisplay.clear_output(wait=True)
env.close()
print("Iterations that were run:",i)

  and should_run_async(code)



step 0 reward:  0  state:  (3, 4)  action= 1
step 1 reward:  -1.0  state:  (3, 4)  action= 1
step 2 reward:  -1.0  state:  (3, 4)  action= 1
step 3 reward:  -1.0  state:  (3, 4)  action= 1
step 4 reward:  -1.0  state:  (3, 4)  action= 1
step 5 reward:  -1.0  state:  (3, 4)  action= 1
step 6 reward:  -1.0  state:  (3, 4)  action= 1
step 7 reward:  -1.0  state:  (3, 4)  action= 1
step 8 reward:  -1.0  state:  (3, 4)  action= 1
step 9 reward:  -1.0  state:  (3, 4)  action= 1
step 10 reward:  -1.0  state:  (3, 4)  action= 1
step 11 reward:  -1.0  state:  (3, 4)  action= 1
step 12 reward:  -1.0  state:  (3, 4)  action= 1
step 13 reward:  -1.0  state:  (3, 4)  action= 1
step 14 reward:  -1.0  state:  (3, 4)  action= 1
step 15 reward:  -1.0  state:  (3, 4)  action= 1
step 16 reward:  -1.0  state:  (3, 4)  action= 1
step 17 reward:  -1.0  state:  (3, 4)  action= 1
step 18 reward:  -1.0  state:  (3, 4)  action= 1
step 19 reward:  -1.0  state:  (3, 4)  action= 1
step 20 reward:  -1.0  state:  (3